In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')

In [4]:
hp_df=pd.read_csv('House Price Data.csv')
y=hp_df['SalePrice']

#Droping output variable from dataset

hp_df.drop(columns=['SalePrice','Id'],inplace=True)

nadf_info=pd.DataFrame({'col_name':hp_df.columns,'na_cnt':hp_df.isnull().sum(),'na_pct':(hp_df.isnull().sum()/hp_df.shape[0])*100})

cols_grt50_ls=list(nadf_info[nadf_info['na_pct']>50]['col_name'])

hp_df.drop(columns=cols_grt50_ls,inplace=True)

x_train,x_test,y_train,y_test=train_test_split(hp_df,y,random_state=49,test_size=0.25)

con_cols=[col for col in hp_df.columns if hp_df[col].dtype=='int64' or hp_df[col].dtype=='float64']

cat_cols=[col for col in hp_df.columns if hp_df[col].dtype=='object']

## filling missing values for contnous columns 
for col in con_cols:
    x_train[col].fillna(x_train[col].mean(),inplace=True)
    x_test[col].fillna(x_train[col].mean(),inplace=True)

## filling missing values for catgorical columns 
    
for col in cat_cols:
    x_train[col].fillna(x_train[col].mode()[0],inplace=True)
    x_test[col].fillna(x_train[col].mode()[0],inplace=True)


In [5]:
scalar=StandardScaler()

for col in con_cols:
    x_train[col]=scalar.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scalar.transform(np.array(x_test[col]).reshape(-1,1))


In [6]:
cat_encd_train= pd.get_dummies(x_train[cat_cols])
cat_encd_test=pd.get_dummies(x_test[cat_cols])

In [7]:
cat_encd_train_final,cat_encd_test_final=cat_encd_train.align(cat_encd_test,join='inner',axis=1)
cat_encd_test_final.shape###aligning train & test data one hot encoded catg columns due to unqual no of columns i.e no of cilumns would differ for that we align to get same

(365, 200)

In [8]:
x_train_final=pd.concat([x_train[con_cols],cat_encd_train_final],axis=1)


In [9]:
x_test_final=pd.concat([x_test[con_cols],cat_encd_test_final],axis=1)

In [10]:
dtree_reg=DecisionTreeRegressor()

In [11]:
dtree_reg.fit(x_train_final,y_train)

DecisionTreeRegressor()

In [12]:
y_test_pred=dtree_reg.predict(x_test_final)
y_test_pred

array([107400., 121500., 149500., 215000.,  85000.,  79500., 187500.,
       107900., 318061., 156500., 132250., 107000., 154000., 224900.,
       171000., 215000., 236000., 112500., 107400., 193500.,  84900.,
       110000., 158000., 335000., 174000., 153000., 187100., 119500.,
       165600., 173500., 205950., 153000., 100000., 134000., 140000.,
        90000.,  83000., 158500., 132000., 129000., 174000.,  55000.,
       224900., 110000., 310000., 195000., 446261.,  90000., 194500.,
       124000., 159500., 200500., 122000., 205000., 144000., 285000.,
       380000., 132000., 212000., 145000., 119000., 101800.,  68500.,
       184000., 339750., 175000., 214000., 205000., 128500.,  86000.,
       181000., 226000., 118000., 167500., 178900., 174900., 100000.,
       250000., 212900., 145000., 168500., 218000., 250000., 179000.,
       260000., 177000., 162500., 165600., 325300.,  67000., 129000.,
       190000., 112000., 140000., 110000., 179600.,  98000., 180000.,
       145000., 3360

In [13]:
dtree_reg.score(x_test_final,y_test)# R2 score for test data

0.8137132567248485

In [18]:
dtree_reg.score(x_train_final,y_train)

1.0

In [17]:
r2_score(y_test,y_test_pred)

0.8137132567248485